# Device Utilities and Common Functions

This notebook contains shared utilities and helper functions that are device-agnostic.
These utilities can be imported and used by other notebooks regardless of the device type.

## Contents
1. Device Capability Checking
2. Resource Monitoring
3. Image Processing Utilities
4. Model Management
5. Performance Profiling

In [ ]:
import sys
sys.path.append('../..')
from base_config import DeviceConfig
from monitoring import DeviceMonitor
import json
import time
from pathlib import Path
import cv2
import numpy as np

class DeviceUtils:
    def __init__(self):
        self.config = DeviceConfig()
        self.monitor = DeviceMonitor()
    
    def check_compatibility(self, notebook_path: str) -> bool:
        """Check if a notebook is compatible with current device."""
        with open(notebook_path, 'r') as f:
            notebook = json.load(f)
            if 'device_requirements' in notebook.get('metadata', {}):
                return self.config.check_notebook_compatibility(
                    notebook['metadata']['device_requirements']
                )
        return True  # No requirements specified

    def profile_execution(self, func, *args, **kwargs):
        """Profile function execution with resource monitoring."""
        start_metrics = self.monitor.collect_metrics()
        start_time = time.time()
        
        result = func(*args, **kwargs)
        
        end_time = time.time()
        end_metrics = self.monitor.collect_metrics()
        
        return {
            'result': result,
            'execution_time': end_time - start_time,
            'memory_delta': end_metrics['memory']['used'] - start_metrics['memory']['used'],
            'cpu_usage': end_metrics['cpu']['usage_percent']
        }

utils = DeviceUtils()


In [ ]:
class ImageUtils:
    @staticmethod
    def optimize_image_size(image, target_size, max_memory_mb=100):
        """Optimize image size based on memory constraints."""
        current_memory = image.nbytes / (1024 * 1024)  # Convert to MB
        if current_memory > max_memory_mb:
            scale_factor = np.sqrt(max_memory_mb / current_memory)
            new_size = tuple(int(dim * scale_factor) for dim in target_size)
            return cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)
        return cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
    
    @staticmethod
    def efficient_normalize(image):
        """Memory-efficient image normalization."""
        image = image.astype(np.float32)
        cv2.normalize(image, image, 0, 1, cv2.NORM_MINMAX)
        return image
    
    @staticmethod
    def create_processing_pipeline(operations):
        """Create a memory-efficient image processing pipeline."""
        def pipeline(image):
            for operation in operations:
                image = operation(image)
                if image is None:
                    raise ValueError(f"Operation {operation.__name__} failed")
            return image
        return pipeline


In [ ]:
class ModelUtils:
    @staticmethod
    def get_model_size(model_path):
        """Get model file size and memory requirements."""
        return Path(model_path).stat().st_size
    
    @staticmethod
    def estimate_memory_requirement(model_path, batch_size=1):
        """Estimate memory requirements for model inference."""
        model_size = ModelUtils.get_model_size(model_path)
        # Rough estimation: model size * 2 for runtime + batch_size factor
        return model_size * 2 * batch_size
    
    @staticmethod
    def check_model_compatibility(model_path):
        """Check if model can run on current device."""
        config = DeviceConfig()
        required_memory = ModelUtils.estimate_memory_requirement(model_path)
        available_memory = config.get_resource_limits()['memory']['available']
        return required_memory < available_memory
